In [1]:
from backend.llm.llm_factory import LLMFactory
import yaml
from pathlib import Path
from pydantic import BaseModel



class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]


config_path = Path("backend/configs/configs.yml")
with config_path.open("r", encoding="utf-8") as f:
    configs = yaml.safe_load(f) or {}

llm_config = configs.get("LLM", {})
llm_client = LLMFactory.create_client(llm_config)

response = llm_client.generate(prompt="Tell me about Canada.", format=Country)
print(type(response))
print(response)


<class 'dict'>
{'name': 'Canada', 'capital': 'Ottawa', 'languages': ['English', 'French']}


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch


model_name = 'NeuML/pubmedbert-base-embeddings'


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

texts = ["This is an example sentence.", "Each sentence is converted into an embedding."]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)


token_embeddings = model_output.last_hidden_state
input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = sum_embeddings / sum_mask


sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)


print("Sentence Embeddings:")
print(sentence_embeddings)
print(f"Shape of embeddings: {sentence_embeddings.shape}")

In [3]:
from backend.encoders.transformer_encoder import TransformerEncoder

encoder = TransformerEncoder(model_name="NeuML/pubmedbert-base-embeddings", device="cuda")
embs = encoder.embed(["This is a test.", "Another sentence."])

len(embs), len(embs[0])  # embs is a CPU torch.Tensor of shape (2, D)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(2, 768)

In [ ]:
from backend.chunkers.semantic_chunking import SemanticChunker
from backend.encoders.transformer_encoder import TransformerEncoder

encoder_model = "intfloat/multilingual-e5-base"
chunker = SemanticChunker(encoder_name="transformer", embedding_model=encoder_model, device="cuda", buffer=0, similarity_threshold=0.9, split_threshold=0.85)
text ="""Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao hơn mức bình thường. Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năng lượng hiệu quả, làm đường tích tụ trong máu. 
Nguyên nhân gây bệnh
Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng lượng. Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.
Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh.
Lối sống: Chế độ ăn uống không lành mạnh và lười vận động có thể dẫn đến tiểu đường, kể cả ở người gầy. 
Biểu hiện phổ biến
Khát nước và đi tiểu nhiều: Cơ thể cố gắng loại bỏ lượng đường dư thừa ra ngoài qua nước tiểu, gây mất nước và cảm giác khát liên tục.
Sụt cân: Mặc dù ăn nhiều nhưng cơ thể vẫn sụt cân vì không thể sử dụng glucose làm năng lượng.
Mệt mỏi kéo dài: Do cơ thể không có đủ năng lượng để hoạt động.
Nhìn mờ: Lượng đường cao trong máu có thể gây tổn thương các mạch máu nhỏ trong mắt.
Tê bì chân tay: Biến chứng thần kinh có thể dẫn đến cảm giác tê bì.
Vết thương lâu lành: Khả năng tự phục hồi của cơ thể bị suy giảm.

Sốt xuất huyết là bệnh truyền nhiễm cấp tính do virus Dengue gây ra, lây lan chủ yếu qua vết đốt của muỗi vằn mang virus. Bệnh có thể có nhiều triệu chứng, từ nhẹ đến nặng, và nguy hiểm nhất là có thể dẫn đến sốc, rối loạn đông máu, suy tạng và tử vong nếu không được điều trị kịp thời. Hiện chưa có thuốc đặc trị và vắc xin phòng bệnh, nên biện pháp phòng ngừa hiệu quả là diệt muỗi, lăng quăng, bọ gậy và phòng tránh muỗi đốt.  
Đặc điểm của bệnh
Nguyên nhân: Virus Dengue, với 4 tuýp huyết thanh DENV-1, DENV-2, DENV-3, DENV-4. 
Cách lây truyền: Qua trung gian là muỗi vằn cái, chủ yếu là loài Aedes aegypti. 
Biểu hiện: Bệnh có thể biểu hiện đa dạng, thường khởi phát đột ngột với sốt cao (40°C), đau đầu, đau hốc mắt, đau cơ, khớp, buồn nôn, nôn, phát ban và sưng hạch bạch huyết. 
Giai đoạn bệnh: Bệnh thường trải qua 3 giai đoạn: giai đoạn sốt, giai đoạn nguy hiểm và giai đoạn hồi phục. 
Biến chứng nguy hiểm: Khi bệnh chuyển sang giai đoạn nguy hiểm có thể gây sốc, chảy máu trong, rối loạn đông máu, suy tạng, trụy tim mạch và dẫn đến tử vong. 
Cách phòng bệnh
Chủ động diệt muỗi, diệt lăng quăng và bọ gậy trong và quanh nhà. 
Tránh để muỗi đốt bằng cách mặc quần áo dài tay, sử dụng màn khi ngủ, và dùng thuốc chống muỗi. 
Phát hiện sớm các triệu chứng và đến cơ sở y tế để được chẩn đoán và điều trị kịp thời. 
"""

chunks = chunker.chunk(text)
print(f"Generated {len(chunks)} chunks:\n")
for i, c in enumerate(chunks, 1):
    print(f"--- Chunk {i} ---")
    print(c)

Generated 8 chunks:

--- Chunk 1 ---
Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao hơn mức bình thường. Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năng lượng hiệu quả, làm đường tích tụ trong máu. Nguyên nhân gây bệnh
Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng lượng. Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu. Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh. Lối sống: Chế độ ăn uống không lành mạnh và lười vận động có thể dẫn đến tiểu đường, kể cả ở người gầy. Biểu hiện phổ biến
Khát nước và đi tiểu nhiều: Cơ thể cố gắng loại bỏ lượng đường dư thừa ra ngoài qua nước tiểu, gây mất nước và cảm giác khát liên tục. Sụt cân: Mặc dù ăn nhiều nhưng cơ thể vẫn sụt cân vì không thể sử dụng glucose làm năng lượng. Mệt mỏi kéo dài: Do cơ thể không có đủ năng 

In [2]:
import re
import numpy as np
from backend.encoders.transformer_encoder import TransformerEncoder
encoder = chunker._get_encoder()
sentences = [s.strip() for s in re.split(r'(?<=[\.\?!])\s+', text.strip()) if s.strip()]
embs = encoder.embed(sentences, batch_size=chunker.batch_size)
embs_np = embs.numpy()
def cosine(a, b):
    an = np.linalg.norm(a)
    bn = np.linalg.norm(b)
    if an == 0 or bn == 0:
        return 0.0
    return float(np.dot(a, b) / (an * bn))
print(f"Total sentences: {len(sentences)}")
for i in range(len(sentences)-1):
    sim = cosine(embs_np[i], embs_np[i+1])
    prefix = sentences[i][:120].replace('\n',' ')
    suffix = sentences[i+1][:120].replace('\n',' ')
    print(f"sim[{i}->{i+1}] = {sim:.4f}  --  {prefix!r}  >>>  {suffix!r}")
boundary_idx = next((i for i,s in enumerate(sentences) if 'Sốt xuất huyết' in s or 'Sốt xuất huyết' in s.replace('\n',' ')), None)
if boundary_idx is not None and boundary_idx>0:
    bsim = cosine(embs_np[boundary_idx-1], embs_np[boundary_idx])
    print('\nDetected paragraph-start sentence index:', boundary_idx)
    print(f'Similarity across paragraph boundary (sentence {boundary_idx-1} -> {boundary_idx}): {bsim:.4f}')
else:
    print('\nCould not detect paragraph boundary sentence by keyword; inspect printed sentences to find it.')

Total sentences: 23
sim[0->1] = 0.8890  --  'Tiểu đường (hay còn gọi là đái tháo đường) là một bệnh rối loạn chuyển hóa, đặc trưng bởi lượng đường trong máu luôn cao'  >>>  'Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năn'
sim[1->2] = 0.9269  --  'Nguyên nhân chính là do cơ thể bị thiếu hụt hoặc đề kháng với insulin, dẫn đến việc không thể chuyển hóa đường thành năn'  >>>  'Nguyên nhân gây bệnh Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng '
sim[2->3] = 0.9020  --  'Nguyên nhân gây bệnh Thiếu hụt hoặc đề kháng insulin: Insulin là hormone giúp đưa glucose từ máu vào tế bào để tạo năng '  >>>  'Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.'
sim[3->4] = 0.8543  --  'Khi insulin không hoạt động tốt, glucose sẽ tích tụ trong máu.'  >>>  'Di truyền: Yếu tố di truyền cũng có thể làm tăng nguy cơ mắc bệnh.'
sim[4->5] = 0.8792  --  'Di truyền: Yếu tố di 

Hãy tìm trên internet các công trình nghiên cứu về việc sử dụng LLM để trích xuất entity từ văn bản y tế.
Sau đó hãy viết class node extractor sử dụng các tài nguyên có sẵn như "backend/llm/ollama_client.py", "backend/encoders/transformer_encoder.py" để trích xuất các entity

Input:
- Văn bản y tế (đoạn văn ngắn về lĩnh vực y tế)
- LLM client: client llm được chỉ định (ollama, gemini,...)
- Model name: model name ứng với client
- Embedding model

Output:
Là một list các entity được trích xuất, mỗi entity bao gồm
- name: tên của entity
- mention: phần văn bản chứa context của entity
- name_embedding: embedding của name
- mention_embedding: ebedding của văn bản mention

Yêu cầu:
- Hoàn thành cài đặt cho class node extraction 
- Nghiên cứu các SOTA cùng topic lựa chọn các prompt tối ưu nhất và áp dụng vào node extraction
- Luôn sử dụng schema (structured output) để lấy dữ liệu output là JSON
- Viết file main.py kết hợp các bước xử lý theo luồng sau:
    1. Load data path từ "data/500_samples_pmc" (giới hạn n sample cho quá trình test)
    2. Load config từ "backend/configs/configs.yml"
    3. Chunking sử dụng section chunking
    4. Trích xuất node